In [15]:
import torch
from ceruleanml.inference import save_fastai_model_state_dict_and_tracing, load_tracing_model, test_tracing_model_one_batch, logits_to_classes
from ceruleanml import data
from ceruleanml import evaluation
from ceruleanml import preprocess
from fastai.data.block import DataBlock
from fastai.vision.data import ImageBlock, MaskBlock
from fastai.data.transforms import RandomSplitter, Normalize
from fastai.metrics import DiceMulti
from ceruleanml.coco_load_fastai import record_collection_to_record_ids, get_image_path, record_to_mask
from datetime import datetime
from pathlib import Path
import os, random
from icevision.visualize import show_data
import torch

tracing_model_path = "../../experiments/cv2/31_May_2022_23_51_32_fastai_unet/tracing_cpu_test_6batch_18_512.pt"

mount_path = "../.."
val_set = "val-no-context-512"
tiled_images_folder_val= "tiled_images_no_context"
json_name_val = "instances_TiledCeruleanDatasetV2NoContextFiles.json"
coco_json_path_val= f"{mount_path}/partitions/{val_set}/{json_name_val}"
tiled_images_folder_val = f"{mount_path}/partitions/{val_set}/{tiled_images_folder_val}"
area_thresh = 10
negative_sample_count_val = 50
record_collection_with_negative_small_filtered_val = preprocess.load_set_record_collection(
    coco_json_path_val, tiled_images_folder_val, area_thresh, negative_sample_count_val
)
record_ids_val = record_collection_to_record_ids(record_collection_with_negative_small_filtered_val)

def get_image_by_record_id(record_id):
    return get_image_path(record_collection_with_negative_small_filtered_val, record_id)

def get_mask_by_record_id(record_id):
    return record_to_mask(record_collection_with_negative_small_filtered_val, record_id)

  0%|          | 0/352 [00:00<?, ?it/s]

100%|██████████| 266/266 [00:00<00:00, 117452.88it/s]


  0%|          | 0/50 [00:00<?, ?it/s]

INFO     - Autofixing records | icevision.parsers.parser:parse:122


  0%|          | 0/50 [00:00<?, ?it/s]

In [16]:
#size = 64  # Progressive resizing could happen here
class_map = {v: k for k, v in data.class_mapping_coco_inv.items()}
class_ints = list(range(1, len(list(class_map.keys())[:-1]) + 1))
mean = [60.73, 190.3, 4.3598]
std = [16.099, 17.846, 9.603]
val_coco_seg_dblock = DataBlock(
    blocks=(ImageBlock, MaskBlock(codes=class_ints)),
    get_x=get_image_by_record_id,
    splitter=RandomSplitter(valid_pct=.9),
    get_y=get_mask_by_record_id,
    batch_tfms=[Normalize.from_stats(mean,std)],
    n_inp=1
)

dls = val_coco_seg_dblock.dataloaders(source=record_ids_val, batch_size=6, n=16)

/root/miniconda3/envs/fastai2/lib/python3.9/site-packages/torch/_tensor.py:1051: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  ret = func(*args, **kwargs)


In [17]:
tracing_model = load_tracing_model(tracing_model_path)
# out_batch_logits = test_tracing_model_one_batch(dls.to('cpu'), tracing_model)
# conf, classes = logits_to_classes(out_batch_logits)

# classes.shape

# conf.shape

# learn.show_results(max_n=4, figsize=(20,20), vmin=0, vmax=3)

Default path for tensorboard logs is `./runs/`

In [ ]:
f'{mount_path}/experiments/cv2/'
!ls '/root/data/experiments/cv2/20_May_2022_19_29_39_fastai_unet'

!ls './runs/'

Copy logs to appropriate exeriments folder in the mounted GCS volume.

In [ ]:
!cp -R './runs/' {modelpath}'/tensorboard/'

Now, run the following from anywhere with gcs authenticated:

In [ ]:
!tensorboard --logdir="./runs"

## Model Inference and Result Evaluation

In [24]:
record_collection_with_negative_small_filtered_val[0].common.height

512

In [29]:
import skimage.io as skio
val_record_ids = record_collection_to_record_ids(record_collection_with_negative_small_filtered_val)
pred_arrs = []
val_arrs = []
for v, i in zip(range(len(record_ids_val[0:10])), record_ids_val[0:10]):
    v = record_collection_with_negative_small_filtered_val[v]
    v_masks = v.detection.masks[0].to_mask(v.common.height,v.common.width).data
    p = get_image_path(record_collection_with_negative_small_filtered_val,i)
    arr = skio.imread(p)
    out_batch_logits = tracing_model(torch.Tensor(np.squeeze(arr))) # squeeze needed for 1 channel input
    conf, class_pred = logits_to_classes(out_batch_logits)
    class_pred = class_pred[0].cpu().detach().numpy()
    pred_arrs.append(class_pred)
    val_arrs.append(v_masks)

RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript, serialized code (most recent call last):
  File "code/__torch__/fastai/vision/models/unet/___torch_mangle_439.py", line 37, in forward
    layers12 = self.layers
    _0 = getattr(layers12, "0")
    _14, _15, _16, _17, _18, _19, _20, _21, _22, _23, _24, _25, _26, _27, _28, _29, _30, _31, _32, _33, _34, = (_0).forward(x, )
                                                                                                                ~~~~~~~~~~~ <--- HERE
    _35 = (_3).forward((_2).forward((_1).forward(_14, ), ), )
    _36 = (_4).forward(_35, _15, _16, _17, _18, _19, )
  File "code/__torch__/torch/nn/modules/container/___torch_mangle_353.py", line 24, in forward
    _1 = getattr(self, "1")
    _0 = getattr(self, "0")
    _8 = (_2).forward((_1).forward((_0).forward(x, ), ), )
                                    ~~~~~~~~~~~ <--- HERE
    _9 = (_4).forward((_3).forward(_8, ), )
    _10 = (

In [ ]:
cm, f1 = eval.cm_f1(val_arrs, pred_arrs, 6, mount_path)

learner = torch.load("/root/data/experiments/cv2/10_May_2022_18_02_59_fastai_unet/18_64_0.493.pkl")

import skimage.io as skio
val_record_ids = record_collection_to_record_ids(valid_records)
pred_arrs = []
with learner.no_logging():
    for i in val_record_ids:
        p = get_image_path(valid_records,i)
        arr = skio.imread(p)
        pred_arr = learner.predict(arr)
        pred_arrs.append(pred_arr)